In [1]:
import sys
import os
# add the root of the project (RecSys) to the Python path
sys.path.append(os.path.abspath('..'))

import yaml
from src.data.data_loader import load_data
from src.data.prepare_data import prepare_data
import pandas as pd
import mlflow
from src.models.model import modeling

In [3]:
# read config
with open('../config/config_inference.yml', 'r') as file:
    config=yaml.load(file, Loader=yaml.SafeLoader)

In [4]:
# load and prepare data
dataframes = load_data(config=config['data_loader'])
dataframes = prepare_data(dataframes=dataframes)

In [5]:
# mlflow tracking
mlflow.set_experiment(experiment_name=config['tracking']['experiment_name'])
mlflow.autolog(disable=True)

with mlflow.start_run(run_name=config['tracking']['run_name']):
    with mlflow.start_run(run_name=config['model']['algo_class'], nested=True):
        # log multiple inputs
        mlflow.set_tag(key='algorithm', value=config['model']['algo_class'])
        mlflow.log_params(params=config['model']['params'])
        mlflow.log_dict(dictionary=config, artifact_file="config_inference.yml")

        # test best algortihm
        model_evaluation=modeling(
            algo_class=config['model']['algo_class']
            , params=config['model']['params']
            )
        model_evaluation.fit(train_set=dataframes['ratings'])
        pred=model_evaluation.inference()
        
        # log model
        mlflow.sklearn.log_model(sk_model=model_evaluation.algo_class, artifact_path=config['model']['algo_class']+'_model')

2024/11/05 19:04:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
